In [1]:
from skimage import exposure
from skimage import feature
import xml.etree.ElementTree as ET
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from pprint import pprint
from voc_eval import parse_rec
from train_img_generator import TrainImageGenerator

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open('datasets/ImageSets/train.txt', 'r') as train_text:
    train_image_indices = train_text.read().strip().split('\n')

training_annotation_path = 'datasets/Annotations/{}.xml'
training_annotations_dict = {
    'waldo': {},
    'wenda': {},
    'wizard': {}
}

for train_image_index in train_image_indices:
    annotation_path = training_annotation_path.format(train_image_index)
    for label in parse_rec(annotation_path):
        target = label['name']
        bbox = label['bbox']
        if train_image_index not in training_annotations_dict[target]:
            training_annotations_dict[target][train_image_index] = [bbox]
        else:
            training_annotations_dict[target][train_image_index].append(bbox)

In [4]:
for target in training_annotations_dict.keys():
    training_image_generator = TrainImageGenerator(target, training_annotations_dict[target])
    training_image_generator.generate_training_images()